# Voice Authentication System

This Python script implements a basic voice authentication system using audio recordings. It allows users to register their voice and later authenticate themselves by comparing their recorded voice with the registered one.


In [ ]:
import os
import numpy as np
import scipy.io.wavfile as wav
from scipy.spatial.distance import euclidean
#from fastdtw import fastdtw
import pyaudio
from sklearn.metrics.pairwise import cosine_similarity


# Audio directory where to save

In [ ]:
USER_VOICE_DIR = "/Users/sushilkumarbarai/Voice_Authentication"


# Recording a audio

In [ ]:
def record_audio(file_path, duration=3, rate=8000, chunk=1024):
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=1,
                        rate=rate, input=True,
                        frames_per_buffer=chunk)
    
    print("Recording...")
    frames = []
    for _ in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)
    
    print("Finished recording.")
    stream.stop_stream()
    stream.close()
    audio.terminate()
    
    wf = wave.open(file_path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()


# Comapre recorded audio with authentication audio (Eculedian distance)

In [ ]:
def compare_audio(audio1, audio2):
    try:
        # audio1='/Users/sushilkumarbarai/Voice_Authentication/sample.wav'
        # audio2='/Users/sushilkumarbarai/Voice_Authentication/sample.wav'
        # Load audio files
        rate1, signal1 = wav.read(audio1)
        rate2, signal2 = wav.read(audio2)

        print(f"Audio rate1 ::: {rate1} signal1 ::: {signal1}")
        print(f"Audio rate2 ::: {rate2} signal2 ::: {signal2}")

        # Ensure both audio signals have the same length
        min_length = min(len(signal1), len(signal2))
        signal1 = signal1[:min_length]
        signal2 = signal2[:min_length]

        # Print the shapes of the signals
        print("Signal 1 shape:", signal1.shape)
        print("Signal 2 shape:", signal2.shape)

        # Convert signals to 1-D arrays
        signal1 = np.array(signal1)
        signal2 = np.array(signal2)

        # Compute Euclidean distance
        distance = np.linalg.norm(signal1 - signal2)

        #Calculate DTW distance between the two audio signals
        # distance,_= fastdtw(signal1, signal2)
        print("Distance:", distance)
        

        return distance

    except Exception as e:
        print("Error occurred during audio comparison:", e)
        return np.inf  # Return infinity distance in case of error


# Authentication a user voice

In [ ]:
def authenticate_user(user_id, new_audio_path):
    user_voice_path = os.path.join(USER_VOICE_DIR, user_id + ".wav")
    if not os.path.exists(user_voice_path):
        print("User voice not found. Please register your voice first.")
        return False
    
    distance = compare_audio(user_voice_path, new_audio_path)
    threshold = 125000
    # Check if the distance is less than the threshold (based on requirment you can adjust)
    if distance < threshold:
        print("Authentication successful!")
        return True
    else:
        print("Authentication failed. Voice does not match.")
        return False


# Execution a code

In [ ]:
def main():
    if not os.path.exists(USER_VOICE_DIR):
        os.makedirs(USER_VOICE_DIR)
    
    user_id = input("Enter your user ID: ")
    user_voice_path = os.path.join(USER_VOICE_DIR, user_id + ".wav")
    
    record_audio(user_voice_path)
    
    while True:
        input("Press Enter to authenticate...")
        new_audio_path = "temp.wav"
        record_audio(new_audio_path)
        
        authenticated = authenticate_user(user_id, new_audio_path)
        if authenticated:
            break


In [ ]:
main()
